In [20]:
from rdflib import Graph, RDF, OWL, Namespace
from rdflib.namespace import RDF, RDFS
from rdflib.term import URIRef

In [ ]:
g = rdflib.Graph()
g.parse("family.ttl")

In [40]:
len(g)

40

In [38]:
def parse(expression):
    expr = expression.replace(" ", "").replace("(", "").replace(")", "")
    l = expr.split("⊔")
    for i, e in enumerate(l):
        l[i] = e.split("⊓")
        for i2, j in enumerate(l[i]):
            if j.startswith("∀"):
                l[i][i2] = {"type": "all", "role": j.removeprefix("∀").split(".")[0] , "name": j.removeprefix("∀").split(".")[1]}
            elif j.startswith("∃"):
                l[i][i2] = {"type": "exists", "role": j.removeprefix("∃").split(".")[0] , "name": j.removeprefix("∃").split(".")[1]}
            elif j.startswith("¬"):
                l[i][i2] = {"type": "not", "name": j.removeprefix("¬")}
            else:
                l[i][i2] = {"type": "atomic", "name" : j}
    return l

In [39]:
def get_uri(g, rdf_term: str) -> URIRef:
        """Helper function that constructs a URI reference object for the RDF term given as str.
        
        Use it when you need to construct a URIRef object, e.g. get_uri("person") to get the URIRef of the concept person.
        
        The given data includes the concepts person, male, female, father, mother; the property hasChild; and eight related individuals.
        """
        return URIRef(dict(g.namespace_manager.namespaces())[""] + rdf_term)

In [42]:
def evaluate(g, expression: str) -> set:
    parsed = parse(expression)
    individuals = {x for x in set(g.subjects(RDF.type, None))if x not in  set(g.subjects(RDF.type, RDFS.Class)) | set(g.subjects(RDF.type, OWL.Class))|set(g.subjects(RDF.type, RDF.Property))}
    for i, d in enumerate(parsed):
        lst = []
        for i2, e in enumerate(d):
            hits = set()
            for individual in individuals:
                if e["type"] == "all":
                    hit = True
                    for _, _, o in g.triples(get_uri(individual), get_uri(e["role"]), None):
                        if o != get_uri(e["name"]):
                            hit = False
                    if hit: 
                        hits.add(individual)
                if e["type"] == "exists":
                    hit = False
                    for _, _, o in g.triples(get_uri(individual), get_uri(e["role"]), None):
                        if o == get_uri(e["name"]):
                            hit = True
                    if hit:
                            hits.add(individual)
                if e["type"] == "not":
                    hit = True
                    for _, _, o in g.triples(get_uri(individual), RDF.type, None):
                        if o == get_uri(e["name"]):
                            hit = False
                    if hit:
                        hits.add(individual)
                if e["type"] == "atomic":
                    hit = False
                    for _, _, o in g.triples(get_uri(individual), RDF.type, None):
                        if o == get_uri(e["name"]):
                            hit = True
                    if hit:
                        hits.add(individual)
            parsed[i][i2] = hits
        parsed[i] = set.intersection(*parsed[i])
    return set.union(parsed)

In [ ]:
evaluate(g, )